# Model for deploying 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
#import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten,Dropout
from keras.optimizers import Adam, RMSprop,SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
from IPython.display import display
from PIL import Image
%matplotlib inline

Using TensorFlow backend.


In [2]:
#Assigning paths
train_path = '../Dataset/train/'
valid_path = '../Dataset/validation/'
test_path = '../Dataset/test/'

In [11]:
train_datagen = ImageDataGenerator(
      
       rotation_range=20,
       width_shift_range=0.2,
       height_shift_range=0.2,
       horizontal_flip=True,
       fill_mode='nearest'
)
 
validation_datagen = ImageDataGenerator()
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 16
test_batchsize = 10
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(256, 256),
        batch_size=train_batchsize,
        classes =['Black_Measles','Blackrot','Isariopsis',"MDB_disease","No_disease","Spyder","mildiou"]
        )
 
validation_generator = validation_datagen.flow_from_directory(
        valid_path,
        target_size=(256, 256),
        batch_size=val_batchsize,
        classes=['Black_Measles','Blackrot','Isariopsis',"MDB_disease","No_disease","Spyder","mildiou"]
        ,
        shuffle=True)
test_generator = validation_datagen.flow_from_directory(test_path,
        target_size=(256, 256),
        batch_size=val_batchsize,
        classes =['Black_Measles','Blackrot','Isariopsis',"MDB_disease","No_disease","Spyder","mildiou"]
        ,
        shuffle=False)

Found 11200 images belonging to 7 classes.
Found 2800 images belonging to 7 classes.
Found 70 images belonging to 7 classes.


In [12]:
from classification_models.keras import Classifiers

In [13]:
from classification_models.keras import Classifiers
from keras.layers import GlobalAveragePooling2D,Flatten
from keras.models import Model
ResNet34, preprocess_input = Classifiers.get('resnet34')

n_classes = 7

# build model
base_model = ResNet34(input_shape=(256,256,3), weights='imagenet', include_top=False)
model =Sequential()
model.add(base_model)
headModel = model.output

headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.25)(headModel)
headModel = Dense(7, activation="softmax")(headModel)
model = Model(inputs=model.input, outputs=headModel)


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_3_input (InputLayer)   (None, 256, 256, 3)       0         
_________________________________________________________________
model_3 (Model)              (None, 8, 8, 512)         21302473  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               16777728  
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

In [15]:
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [16]:
# Save the model according to the conditions  
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

checkpoint = ModelCheckpoint("ios.h5", monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, mode='auto')


In [17]:
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,epochs=5,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1, callbacks=([early,checkpoint]))

Epoch 1/5
700/700 [==============================] - 262s 374ms/step - loss: 0.7940 - acc: 0.7290 - val_loss: 0.2966 - val_acc: 0.8896

Epoch 00001: loss improved from inf to 0.79405, saving model to ios.h5
Epoch 2/5
700/700 [==============================] - 259s 370ms/step - loss: 0.2842 - acc: 0.9042 - val_loss: 0.1816 - val_acc: 0.9375

Epoch 00002: loss improved from 0.79405 to 0.28421, saving model to ios.h5
Epoch 3/5
700/700 [==============================] - 264s 377ms/step - loss: 0.2013 - acc: 0.9321 - val_loss: 0.1857 - val_acc: 0.9450

Epoch 00003: loss improved from 0.28421 to 0.20125, saving model to ios.h5
Epoch 4/5
700/700 [==============================] - 265s 379ms/step - loss: 0.1690 - acc: 0.9459 - val_loss: 0.1424 - val_acc: 0.9557

Epoch 00004: loss improved from 0.20125 to 0.16897, saving model to ios.h5
Epoch 5/5
700/700 [==============================] - 258s 369ms/step - loss: 0.1429 - acc: 0.9531 - val_loss: 0.1039 - val_acc: 0.9661

Epoch 00005: loss improv

# Converting keras model to coreml

In [19]:
import coremltools
#from keras.models import load_model
#model = load_model('../saved_models/resnet18.h5')
coreml_model = coremltools.converters.keras.convert(model, input_names=['image'], image_input_names='image', class_labels=['Black_Measles','Blackrot','Isariopsis','MDB_disease','No_disease','Spyder','mildiou'])
coreml_model.save('ios2.mlmodel')

0 : model_3_input, <keras.engine.input_layer.InputLayer object at 0x7fb283813b38>
1 : model_3_bn_data, <keras.layers.normalization.BatchNormalization object at 0x7fb2886785c0>
2 : model_3_zero_padding2d_35, <keras.layers.convolutional.ZeroPadding2D object at 0x7fb2886787f0>
3 : model_3_conv0, <keras.layers.convolutional.Conv2D object at 0x7fb26c0beda0>
4 : model_3_bn0, <keras.layers.normalization.BatchNormalization object at 0x7fb26df4ea90>
5 : model_3_relu0, <keras.layers.core.Activation object at 0x7fb26df4ecc0>
6 : model_3_zero_padding2d_36, <keras.layers.convolutional.ZeroPadding2D object at 0x7fb26df72a58>
7 : model_3_pooling0, <keras.layers.pooling.MaxPooling2D object at 0x7fb26df31358>
8 : model_3_stage1_unit1_bn1, <keras.layers.normalization.BatchNormalization object at 0x7fb26deb8f28>
9 : model_3_stage1_unit1_relu1, <keras.layers.core.Activation object at 0x7fb26deb8e10>
10 : model_3_zero_padding2d_37, <keras.layers.convolutional.ZeroPadding2D object at 0x7fb26de25240>
11 : mo

91 : model_3_add_25, <keras.layers.merge.Add object at 0x7fb26c344160>
92 : model_3_stage3_unit3_bn1, <keras.layers.normalization.BatchNormalization object at 0x7fb26c344208>
93 : model_3_stage3_unit3_relu1, <keras.layers.core.Activation object at 0x7fb26c3442b0>
94 : model_3_zero_padding2d_55, <keras.layers.convolutional.ZeroPadding2D object at 0x7fb26c3a4cc0>
95 : model_3_stage3_unit3_conv1, <keras.layers.convolutional.Conv2D object at 0x7fb26c39c630>
96 : model_3_stage3_unit3_bn2, <keras.layers.normalization.BatchNormalization object at 0x7fb26c3ed278>
97 : model_3_stage3_unit3_relu2, <keras.layers.core.Activation object at 0x7fb26c3ed208>
98 : model_3_zero_padding2d_56, <keras.layers.convolutional.ZeroPadding2D object at 0x7fb26c3ed0f0>
99 : model_3_stage3_unit3_conv2, <keras.layers.convolutional.Conv2D object at 0x7fb26c3d9a20>
100 : model_3_add_26, <keras.layers.merge.Add object at 0x7fb26c418550>
101 : model_3_stage3_unit4_bn1, <keras.layers.normalization.BatchNormalization obje